In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import ta


In [ ]:

# Define the list of stocks
stocks = ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS', 'HINDUNILVR.NS', 'HDFC.NS', 'KOTAKBANK.NS', 'BHARTIARTL.NS', 'LT.NS',
    'ITC.NS', 'ASIANPAINT.NS', 'SBIN.NS', 'BAJFINANCE.NS', 'HCLTECH.NS', 'AXISBANK.NS', 'DMART.NS', 'MARUTI.NS', 'SUNPHARMA.NS', 'ADANIGREEN.NS',
    'ONGC.NS', 'NTPC.NS', 'POWERGRID.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'ULTRACEMCO.NS', 'GRASIM.NS', 'BAJAJFINSV.NS', 'BRITANNIA.NS', 'ADANIPORTS.NS',
    'HEROMOTOCO.NS', 'WIPRO.NS', 'DRREDDY.NS', 'HDFCLIFE.NS', 'TECHM.NS', 'SBICARD.NS', 'SHREECEM.NS', 'CIPLA.NS', 'NESTLEIND.NS', 'DABUR.NS',
    'VEDL.NS', 'INDUSINDBK.NS', 'HINDZINC.NS', 'BOSCHLTD.NS', 'COALINDIA.NS', 'PIDILITIND.NS', 'DIVISLAB.NS', 'ADANIENT.NS', 'BAJAJHLDNG.NS', 'LUPIN.NS',
    'ICICIPRULI.NS', 'APOLLOHOSP.NS', 'COLPAL.NS', 'GODREJCP.NS', 'IOC.NS', 'M&M.NS', 'NAUKRI.NS', 'HAVELLS.NS', 'PGHH.NS', 'EICHERMOT.NS',
    'BPCL.NS', 'SIEMENS.NS', 'BIOCON.NS', 'TATAPOWER.NS', 'BERGEPAINT.NS', 'SRF.NS', 'CONCOR.NS', 'AMBUJACEM.NS', 'GLAND.NS', 'INDIGO.NS',
    'MANAPPURAM.NS', 'MCDOWELL-N.NS', 'LICI.NS', 'HDFCAMC.NS', 'DLF.NS', 'BANDHANBNK.NS', 'AUROPHARMA.NS', 'MPHASIS.NS',
    'TORNTPHARM.NS', 'TRENT.NS', 'PETRONET.NS', 'CUB.NS', 'GMRINFRA.NS', 'BAJAJ-AUTO.NS', 'ABBOTINDIA.NS', 'NMDC.NS', 'TITAN.NS', 'BEL.NS',
    'HINDALCO.NS', 'VOLTAS.NS', 'MUTHOOTFIN.NS', 'BEL.NS', 'EXIDEIND.NS', 'NATIONALUM.NS', 'BANKBARODA.NS', 'SAIL.NS', 'HINDCOPPER.NS', 'SUNTV.NS']


In [ ]:

# Define the date range
end_date = datetime.now()
start_date = end_date - timedelta(days=5 * 365)


In [ ]:

# Fetch historical data and financial metrics
def fetch_data(stocks, start_date, end_date):
    all_data = {}
    for stock in stocks:
        try:
            df = yf.download(stock, start=start_date, end=end_date)
            info = yf.Ticker(stock).info
            df['PE_Ratio'] = info.get('trailingPE', np.nan)
            df['Earnings'] = info.get('trailingEps', np.nan)
            all_data[stock] = df
        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
    return all_data


In [ ]:

stock_data = fetch_data(stocks, start_date, end_date)


In [ ]:

# Add features using technical indicators and financial metrics
def add_features(df):
    df['RSI'] = ta.momentum.rsi(df['Close'])
    df['MACD'] = ta.trend.macd_diff(df['Close'])
    bollinger = ta.volatility.BollingerBands(df['Close'])
    df['BB_High'] = bollinger.bollinger_hband()
    df['BB_Low'] = bollinger.bollinger_lband()
    df['Price_Lag1'] = df['Close'].shift(1)
    df['Price_Lag2'] = df['Close'].shift(2)
    df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)
    return df.dropna()



In [ ]:
for stock, df in stock_data.items():
    stock_data[stock] = add_features(df)


In [ ]:

# Split data into training and testing sets
def train_test_split(df, test_size=0.2):
    split_index = int(len(df) * (1 - test_size))
    train_data = df[:split_index]
    test_data = df[split_index:]
    return train_data, test_data


In [ ]:

train_data = {}
test_data = {}


In [ ]:

for stock, df in stock_data.items():
    train_data[stock], test_data[stock] = train_test_split(df)


In [ ]:

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)


In [ ]:

def prepare_data(df):
    features = df[['RSI', 'MACD', 'BB_High', 'BB_Low', 'Price_Lag1', 'Price_Lag2', 'PE_Ratio', 'Earnings']]
    target = df['Target']
    return features, target


In [ ]:

reports = []


In [ ]:

for stock in stocks:
    train_features, train_target = prepare_data(train_data[stock])
    test_features, test_target = prepare_data(test_data[stock])

    if train_features.empty or not train_target.any():
        print(f"Training data for {stock} is empty or contains no positive samples. Skipping model fitting.")
        continue  # Skip model fitting for this stock

    try:
        model.fit(train_features, train_target)
        predictions = model.predict(test_features)

        if not predictions.any():
            print(f"No predictions made for {stock}. Skipping performance evaluation.")
            continue  # Skip performance evaluation for this stock

        accuracy = accuracy_score(test_target, predictions)
        precision = precision_score(test_target, predictions)
        recall = recall_score(test_target, predictions)

        print(f"Model performance for {stock}:")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print("=" * 40)

        reports.append({
            'Stock': stock,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall
        })

    except Exception as e:
        print(f"Error while training model for {stock}: {e}")


In [ ]:

# Generate trading signals
def generate_trading_signals(df, model):
    features, _ = prepare_data(df)
    df['Prediction'] = model.predict(features)
    df['Buy_Signal'] = (df['Prediction'] == 1) & (df['Prediction'].shift(1) == 0)
    df['Sell_Signal'] = (df['Prediction'] == 0) & (df['Prediction'].shift(1) == 1)
    return df


In [ ]:

for stock in stocks:
    try:
        test_data[stock] = generate_trading_signals(test_data[stock], model)
    except Exception as e:
        print(f"Error while generating trading signals for {stock}: {e}")


In [ ]:

# Implement a backtesting framework
def backtest(df, initial_investment=10000):
    cash = initial_investment
    position = 0
    portfolio_value = []
    trade_log = []

    for i in range(len(df)):
        if df['Buy_Signal'].iloc[i]:
            if cash > 0:
                position = cash / df['Close'].iloc[i]
                cash = 0
                trade_log.append(('Buy', df.index[i], df['Close'].iloc[i]))
        elif df['Sell_Signal'].iloc[i]:
            if position > 0:
                cash = position * df['Close'].iloc[i]
                position = 0
                trade_log.append(('Sell', df.index[i], df['Close'].iloc[i]))
        portfolio_value.append(cash + position * df['Close'].iloc[i])

    df['Portfolio_Value'] = portfolio_value
    return df, trade_log


In [ ]:

performance_reports = []


In [ ]:

for stock in stocks:
    try:
        train_features, train_target = prepare_data(train_data[stock])
        test_features, test_target = prepare_data(test_data[stock])

        if train_features.empty or not train_target.any():
            print(f"Training data for {stock} is empty or contains no positive samples. Skipping model fitting.")
            continue  # Skip model fitting for this stock

        model.fit(train_features, train_target)
        predictions = model.predict(test_features)

        if not predictions.any():
            print(f"No predictions made for {stock}. Skipping performance evaluation.")
            continue  # Skip performance evaluation for this stock

        accuracy = accuracy_score(test_target, predictions)
        precision = precision_score(test_target, predictions)
        recall = recall_score(test_target, predictions)

        print(f"Model performance for {stock}:")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print("=" * 40)

        performance_reports.append({
            'Stock': stock,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall
        })

    except Exception as e:
        print(f"Error while evaluating performance for {stock}: {e}")


In [ ]:

# Generate performance report
def summary_report(reports):
    summary = pd.DataFrame(reports)
    return summary


In [ ]:

summary = summary_report(performance_reports)
print(summary)


In [ ]:

# Write summary report
def write_summary(summary):
    print("Performance Summary Report:")
    print(summary)
    print("=" * 40)
    for _, row in summary.iterrows():
        print(f"Stock: {row['Stock']}")
        print(f"Accuracy: {row['Accuracy']:.2%}")
        print(f"Precision: {row['Precision']:.2%}")
        print(f"Recall: {row['Recall']:.2%}")
        print("=" * 40)


In [ ]:

write_summary(summary)


In [ ]:

# Visualize buy/sell signals and portfolio value
def visualize(stock, df):
    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['Close'], label='Close Price')

    buy_indices = df.index[df['Buy_Signal']]
    sell_indices = df.index[df['Sell_Signal']]

    plt.scatter(buy_indices, df['Close'][df['Buy_Signal']], label='Buy Signal', marker='^', color='green')
    plt.scatter(sell_indices, df['Close'][df['Sell_Signal']], label='Sell Signal', marker='v', color='red')

    plt.title(f'{stock} Price with Buy/Sell Signals')
    plt.legend()
    plt.show()

    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['Portfolio_Value'], label='Portfolio Value')
    plt.title(f'{stock} Portfolio Value Over Time')
    plt.legend()
    plt.show()


In [ ]:

for stock in stocks:
    try:
        visualize(stock, test_data[stock])
    except Exception as e:
        print(f"Error while visualizing {stock}: {e}")


In [ ]:

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

def prepare_data(df):
    features = df[['RSI', 'MACD', 'BB_High', 'BB_Low', 'Price_Lag1', 'Price_Lag2', 'PE_Ratio', 'Earnings']]
    target = df['Target']
    return features, target

reports = []

for stock in stocks:
    train_features, train_target = prepare_data(train_data[stock])
    test_features, test_target = prepare_data(test_data[stock])

    if train_features.empty or not train_target.any():
        print(f"Training data for {stock} is empty or contains no positive samples. Skipping model fitting.")
        continue  # Skip model fitting for this stock

    try:
        model.fit(train_features, train_target)
        predictions = model.predict(test_features)

        if not predictions.any():
            print(f"No predictions made for {stock}. Skipping performance evaluation.")
            continue  # Skip performance evaluation for this stock

        accuracy = accuracy_score(test_target, predictions)
        precision = precision_score(test_target, predictions)
        recall = recall_score(test_target, predictions)

        print(f"Model performance for {stock}:")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print("=" * 40)

        reports.append({
            'Stock': stock,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall
        })

    except Exception as e:
        print(f"Error while training model for {stock}: {e}")


In [ ]:

# Generate trading signals
def generate_trading_signals(df, model):
    features, _ = prepare_data(df)
    df['Prediction'] = model.predict(features)
    df['Buy_Signal'] = (df['Prediction'] == 1) & (df['Prediction'].shift(1) == 0)
    df['Sell_Signal'] = (df['Prediction'] == 0) & (df['Prediction'].shift(1) == 1)
    return df


In [ ]:

for stock in stocks:
    try:
        test_data[stock] = generate_trading_signals(test_data[stock], model)
    except Exception as e:
        print(f"Error while generating trading signals for {stock}: {e}")


In [ ]:

# Implement a backtesting framework
def backtest(df, initial_investment=10000):
    cash = initial_investment
    position = 0
    portfolio_value = []
    trade_log = []

    for i in range(len(df)):
        if df['Buy_Signal'].iloc[i]:
            if cash > 0:
                position = cash / df['Close'].iloc[i]
                cash = 0
                trade_log.append(('Buy', df.index[i], df['Close'].iloc[i]))
        elif df['Sell_Signal'].iloc[i]:
            if position > 0:
                cash = position * df['Close'].iloc[i]
                position = 0
                trade_log.append(('Sell', df.index[i], df['Close'].iloc[i]))
        portfolio_value.append(cash + position * df['Close'].iloc[i])

    df['Portfolio_Value'] = portfolio_value
    return df, trade_log


In [ ]:

performance_reports = []


In [ ]:

for stock in stocks:
    try:
        train_features, train_target = prepare_data(train_data[stock])
        test_features, test_target = prepare_data(test_data[stock])

        if train_features.empty or not train_target.any():
            print(f"Training data for {stock} is empty or contains no positive samples. Skipping model fitting.")
            continue  # Skip model fitting for this stock

        model.fit(train_features, train_target)
        predictions = model.predict(test_features)

        if not predictions.any():
            print(f"No predictions made for {stock}. Skipping performance evaluation.")
            continue  # Skip performance evaluation for this stock

        accuracy = accuracy_score(test_target, predictions)
        precision = precision_score(test_target, predictions)
        recall = recall_score(test_target, predictions)

        print(f"Model performance for {stock}:")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print("=" * 40)

        performance_reports.append({
            'Stock': stock,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall
        })

    except Exception as e:
        print(f"Error while evaluating performance for {stock}: {e}")


In [ ]:

# Generate performance report
def summary_report(reports):
    summary = pd.DataFrame(reports)
    return summary


In [ ]:

summary = summary_report(performance_reports)
print(summary)


In [ ]:

# Write summary report
def write_summary(summary):
    print("Performance Summary Report:")
    print(summary)
    print("=" * 40)
    for _, row in summary.iterrows():
        print(f"Stock: {row['Stock']}")
        print(f"Accuracy: {row['Accuracy']:.2%}")
        print(f"Precision: {row['Precision']:.2%}")
        print(f"Recall: {row['Recall']:.2%}")
        print("=" * 40)


In [ ]:

write_summary(summary)


In [ ]:

# Visualize buy/sell signals and portfolio value
def visualize(stock, df):
    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['Close'], label='Close Price')

    buy_indices = df.index[df['Buy_Signal']]
    sell_indices = df.index[df['Sell_Signal']]

    plt.scatter(buy_indices, df['Close'][df['Buy_Signal']], label='Buy Signal', marker='^', color='green')
    plt.scatter(sell_indices, df['Close'][df['Sell_Signal']], label='Sell Signal', marker='v', color='red')

    plt.title(f'{stock} Price with Buy/Sell Signals')
    plt.legend()
    plt.show()

    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['Portfolio_Value'], label='Portfolio Value')
    plt.title(f'{stock} Portfolio Value Over Time')
    plt.legend()
    plt.show()


In [ ]:

for stock in stocks:
    try:
        visualize(stock, test_data[stock])
    except Exception as e:
        print(f"Error while visualizing {stock}: {e}")


In [ ]:

train_data = {}
test_data = {}


In [ ]:

for stock, df in stock_data.items():
    train_data[stock], test_data[stock] = train_test_split(df)


In [ ]:

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)


In [ ]:

def prepare_data(df):
    features = df[['RSI', 'MACD', 'BB_High', 'BB_Low', 'Price_Lag1', 'Price_Lag2', 'PE_Ratio', 'Earnings']]
    target = df['Target']
    return features, target


In [ ]:

reports = []


In [ ]:

for stock in stocks:
    train_features, train_target = prepare_data(train_data[stock])
    test_features, test_target = prepare_data(test_data[stock])

    if train_features.empty or not train_target.any():
        print(f"Training data for {stock} is empty or contains no positive samples. Skipping model fitting.")
        continue  # Skip model fitting for this stock

    try:
        model.fit(train_features, train_target)
        predictions = model.predict(test_features)

        if not predictions.any():
            print(f"No predictions made for {stock}. Skipping performance evaluation.")
            continue  # Skip performance evaluation for this stock

        accuracy = accuracy_score(test_target, predictions)
        precision = precision_score(test_target, predictions)
        recall = recall_score(test_target, predictions)

        print(f"Model performance for {stock}:")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print("=" * 40)

        reports.append({
            'Stock': stock,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall
        })

    except Exception as e:
        print(f"Error while training model for {stock}: {e}")


In [ ]:

# Generate trading signals
def generate_trading_signals(df, model):
    features, _ = prepare_data(df)
    df['Prediction'] = model.predict(features)
    df['Buy_Signal'] = (df['Prediction'] == 1) & (df['Prediction'].shift(1) == 0)
    df['Sell_Signal'] = (df['Prediction'] == 0) & (df['Prediction'].shift(1) == 1)
    return df


In [ ]:

for stock in stocks:
    try:
        test_data[stock] = generate_trading_signals(test_data[stock], model)
    except Exception as e:
        print(f"Error while generating trading signals for {stock}: {e}")


In [ ]:

# Implement a backtesting framework
def backtest(df, initial_investment=10000):
    cash = initial_investment
    position = 0
    portfolio_value = []
    trade_log = []

    for i in range(len(df)):
        if df['Buy_Signal'].iloc[i]:
            if cash > 0:
                position = cash / df['Close'].iloc[i]
                cash = 0
                trade_log.append(('Buy', df.index[i], df['Close'].iloc[i]))
        elif df['Sell_Signal'].iloc[i]:
            if position > 0:
                cash = position * df['Close'].iloc[i]
                position = 0
                trade_log.append(('Sell', df.index[i], df['Close'].iloc[i]))
        portfolio_value.append(cash + position * df['Close'].iloc[i])

    df['Portfolio_Value'] = portfolio_value
    return df, trade_log


In [ ]:

performance_reports = []


In [ ]:

for stock in stocks:
    try:
        train_features, train_target = prepare_data(train_data[stock])
        test_features, test_target = prepare_data(test_data[stock])

        if train_features.empty or not train_target.any():
            print(f"Training data for {stock} is empty or contains no positive samples. Skipping model fitting.")
            continue  # Skip model fitting for this stock

        model.fit(train_features, train_target)
        predictions = model.predict(test_features)

        if not predictions.any():
            print(f"No predictions made for {stock}. Skipping performance evaluation.")
            continue  # Skip performance evaluation for this stock

        accuracy = accuracy_score(test_target, predictions)
        precision = precision_score(test_target, predictions)
        recall = recall_score(test_target, predictions)

        print(f"Model performance for {stock}:")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print("=" * 40)

        performance_reports.append({
            'Stock': stock,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall
        })

    except Exception as e:
        print(f"Error while evaluating performance for {stock}: {e}")


In [ ]:

# Generate performance report
def summary_report(reports):
    summary = pd.DataFrame(reports)
    return summary


In [ ]:

summary = summary_report(performance_reports)
print(summary)


In [ ]:

# Write summary report
def write_summary(summary):
    print("Performance Summary Report:")
    print(summary)
    print("=" * 40)
    for _, row in summary.iterrows():
        print(f"Stock: {row['Stock']}")
        print(f"Accuracy: {row['Accuracy']:.2%}")
        print(f"Precision: {row['Precision']:.2%}")
        print(f"Recall: {row['Recall']:.2%}")
        print("=" * 40)


In [ ]:

write_summary(summary)


In [ ]:

# Visualize buy/sell signals and portfolio value
def visualize(stock, df):
    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['Close'], label='Close Price')

    buy_indices = df.index[df['Buy_Signal']]
    sell_indices = df.index[df['Sell_Signal']]

    plt.scatter(buy_indices, df['Close'][df['Buy_Signal']], label='Buy Signal', marker='^', color='green')
    plt.scatter(sell_indices, df['Close'][df['Sell_Signal']], label='Sell Signal', marker='v', color='red')

    plt.title(f'{stock} Price with Buy/Sell Signals')
    plt.legend()
    plt.show()

    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['Portfolio_Value'], label='Portfolio Value')
    plt.title(f'{stock} Portfolio Value Over Time')
    plt.legend()
    plt.show()


In [ ]:

for stock in stocks:
    try:
        visualize(stock, test_data[stock])
    except Exception as e:
        print(f"Error while visualizing {stock}: {e}")
